# This notebook is a test bed to check the functionality of the timeloop package and its efficacy for deployment of the Muskometer backend. 

## I'm going to need to have a background script running that scans for new tweets periodically and updates stock data periodically as well.  In addition, when new tweets are found, they need to be put through the anomaly detection alogrithm and the datasets used by the front end need to be updated.  The timeloop package seems to be a good way to do all this simultaneously within a single script.

In [1]:
import time
import timeloop
import datetime
from collections import defaultdict
import os, sys
import pandas as pd
import GetOldTweets3 as got
from timeloop import Timeloop
from datetime import timedelta

In [2]:
# This example code is shamelessly copied from :
# https://medium.com/greedygame-engineering/an-elegant-way-to-run-periodic-tasks-in-python-61b7c477b679
tl = Timeloop()

@tl.job(interval=timedelta(seconds=2))
def sample_job_every_2s():
    print ("2s job current time : {}".format(time.ctime()))
    
@tl.job(interval=timedelta(seconds=5))
def sample_job_every_5s():
    print ("5s job current time : {}".format(time.ctime()))
    
@tl.job(interval=timedelta(seconds=10))
def sample_job_every_10s():
    print ("10s job current time : {}".format(time.ctime()))
    
if __name__ == "__main__":
    tl.start(block=True)

[2020-06-28 13:36:28,636] [timeloop] [INFO] Starting Timeloop..
[2020-06-28 13:36:28,638] [timeloop] [INFO] Registered job <function sample_job_every_2s at 0x7fc4d7e15310>
[2020-06-28 13:36:28,639] [timeloop] [INFO] Registered job <function sample_job_every_5s at 0x7fc4d88040d0>
[2020-06-28 13:36:28,640] [timeloop] [INFO] Registered job <function sample_job_every_10s at 0x7fc4d88041f0>
[2020-06-28 13:36:28,640] [timeloop] [INFO] Timeloop now started. Jobs will run based on the interval set
2s job current time : Sun Jun 28 13:36:30 2020
2s job current time : Sun Jun 28 13:36:32 2020
5s job current time : Sun Jun 28 13:36:33 2020
2s job current time : Sun Jun 28 13:36:34 2020
2s job current time : Sun Jun 28 13:36:36 2020
10s job current time : Sun Jun 28 13:36:38 2020
5s job current time : Sun Jun 28 13:36:38 2020
2s job current time : Sun Jun 28 13:36:38 2020
2s job current time : Sun Jun 28 13:36:40 2020
2s job current time : Sun Jun 28 13:36:42 2020
5s job current time : Sun Jun 28 1

## Looks like that worked as advertised!

### Let's start building up the functionality of the backend and test it as we go.  First I'll load up the stored tweets from Elon Musk, scan for new tweets, and add them to the record.

In [3]:
def reload_tweet_data(path,username="elonmusk"):
   #note we'll have to do a .drop and set the 'Time' column to the proper values every time
    df = pd.read_csv(path+username+'_tweets.csv').drop(['Unnamed: 0'],axis='columns')
    #order by earliest first
    df['Time'] = pd.to_datetime(df['Time'])#.sort_values(by='Time',ascending=True)
    return df.sort_values(by='Time',ascending=True).reset_index().drop('index',axis='columns')
def prepend_new_tweets(df_new,df_old): #adds the new tweets to the front of the data set and resets the index
    result = pd.concat([df_old,df_new]).reset_index().drop('index',axis='columns')
    #makes sure we're sorted properly in time order
    result.sort_values(by='Time',ascending=True)
    return result.reset_index().drop('index',axis='columns')
def store_tweet_data(df,path,username="elonmusk"):
    df.to_csv(path+username+'_tweets.csv')
    return
def scan_for_new_tweets(path,username="elonmusk"):
    df_old = reload_tweet_data(path,username) #get the old tweets
    #look for new tweets starting from latest date
    df_new = scrape_new_tweets(df_old['Time'].max(),username)
    if df_new == None:# No new tweets
        return df_old
    else:
        df_combined = prepend_new_tweets(df_new,df_old)
        return df_combined
def scrape_new_tweets(t_last_tweet,username = "elonmusk"):
    """Function to scrape the recent tweets of Elon Musk"""
    #t_last_tweet must be pandas Timestamp data
    os.makedirs('tweet_data', exist_ok=True)
    date_str = str(t_last_tweet.date().year)+"-"\
              +str(t_last_tweet.date().month)+"-"\
              +str(t_last_tweet.date().day)
    count = 0
    # Creation of query object                                                                                                                                                                                      
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                                               .setMaxTweets(count)\
                                               .setSince(date_str)
    # Creation of list that contains all tweets                                                                                                                                                                     
    tweets = None
    for ntries in range(5):
        try:
            tweets = got.manager.TweetManager.getTweets(tweetCriteria)
        except SystemExit:
            print("Trying again in 15 minutes.")
            time.sleep(15*60)
        else:
            break
    if tweets is None:
        print("Failed after 5 tries, quitting!")
        exit(1)

    data = defaultdict(list)
    for t in tweets:
        data["username"].append(username)
        data["tweet_id"].append(t.id)
        data["reply_to"].append(t.to)
        data["date"].append(t.date)
        data["retweets"].append(t.retweets)
        data["favorites"].append(t.favorites)
        data["hashtags"].append(list(set(t.hashtags.split())))
        data["mentions"].append(t.mentions)
        data["text"].append(t.text)
        data["permalink"].append(t.permalink)
    if len(data) == 0: #no new tweets
        return None
    else:
        #make a DataFrame out of the scraped tweets
        df = pd.DataFrame(data, columns=["username","tweet_id",
                                         "reply_to","date","retweets",
                                         "favorites","hashtags","mentions",
                                         "text","permalink"])        
        # Convert 'Time' column to datetime and strip time information.
        df['Time'] = pd.to_datetime(df['date'])
        return df.sort_values(by='Time',ascending=True)

In [4]:
elon_tweets_df = reload_tweet_data('../data/raw/',username="elonmusk")

In [5]:
store_tweet_data(elon_tweets_df,'../data/raw/',username="elonmusk")

In [6]:
elon_tweets_df.tail()

,username,tweet_id,reply_to,retweets,favorites,hashtags,mentions,text,permalink,Time
9801,elonmusk,1267146619562201090,SpaceX,5576,67423,[],@Space_Station,Congratulations Bob & Doug on docking & hatch ...,https://twitter.com/elonmusk/status/1267146619...,2020-05-31 17:31:11+00:00
9802,elonmusk,1267156817295085575,Rogozin,1209,7558,[],NaN,"Спасибо, сэр, ха-ха. Мы рассчитываем на взаимо...",https://twitter.com/elonmusk/status/1267156817...,2020-05-31 18:11:42+00:00
9803,elonmusk,1267157474886455296,NASASpaceflight,708,14436,[],NaN,Brought home by same person who placed it ther...,https://twitter.com/elonmusk/status/1267157474...,2020-05-31 18:14:19+00:00
9804,elonmusk,1267160409498357764,NASASpaceflight,81,2494,[],NaN,Must be due to relativistic aging,https://twitter.com/elonmusk/status/1267160409...,2020-05-31 18:25:58+00:00
9805,elonmusk,1267180654896254976,SpaceX,22581,250519,[],NaN,Nine years later,https://twitter.com/elonmusk/status/1267180654...,2020-05-31 19:46:25+00:00


### Okay, let's see if we can scrape the new tweets from June.

In [7]:
new_tweets_df = scrape_new_tweets(elon_tweets_df['Time'].max(),username = "elonmusk")

In [8]:
new_tweets_df.tail()

,username,tweet_id,reply_to,date,retweets,favorites,hashtags,mentions,text,permalink,Time
4,elonmusk,1276716075422367746,waitbutwhy,2020-06-27 03:16:47+00:00,820,12922,[],,It’s a tough one. That’s why we should expand ...,https://twitter.com/elonmusk/status/1276716075...,2020-06-27 03:16:47+00:00
3,elonmusk,1276738326804873217,slashdot,2020-06-27 04:45:12+00:00,136,2263,[],,"Verified should be far more widespread, simply...",https://twitter.com/elonmusk/status/1276738326...,2020-06-27 04:45:12+00:00
2,elonmusk,1276747755914842112,CathieDWood,2020-06-27 05:22:40+00:00,104,2713,[],,For sure,https://twitter.com/elonmusk/status/1276747755...,2020-06-27 05:22:40+00:00
1,elonmusk,1276957305842528256,BLKMDL3,2020-06-27 19:15:21+00:00,324,15549,[],,Major Supercharger increases are underway for ...,https://twitter.com/elonmusk/status/1276957305...,2020-06-27 19:15:21+00:00
0,elonmusk,1276959792876011520,GerberKawasaki,2020-06-27 19:25:14+00:00,475,6698,[],,"Physics favors electric transport, batteries f...",https://twitter.com/elonmusk/status/1276959792...,2020-06-27 19:25:14+00:00


In [9]:
#if no new tweets, we should get back a None data type
test_tweets_df = scrape_new_tweets(new_tweets_df['Time'].max(),username = "elonmusk")

In [10]:
test_tweets_df.head()

,username,tweet_id,reply_to,date,retweets,favorites,hashtags,mentions,text,permalink,Time
4,elonmusk,1276716075422367746,waitbutwhy,2020-06-27 03:16:47+00:00,820,12922,[],,It’s a tough one. That’s why we should expand ...,https://twitter.com/elonmusk/status/1276716075...,2020-06-27 03:16:47+00:00
3,elonmusk,1276738326804873217,slashdot,2020-06-27 04:45:12+00:00,136,2263,[],,"Verified should be far more widespread, simply...",https://twitter.com/elonmusk/status/1276738326...,2020-06-27 04:45:12+00:00
2,elonmusk,1276747755914842112,CathieDWood,2020-06-27 05:22:40+00:00,104,2713,[],,For sure,https://twitter.com/elonmusk/status/1276747755...,2020-06-27 05:22:40+00:00
1,elonmusk,1276957305842528256,BLKMDL3,2020-06-27 19:15:21+00:00,324,15550,[],,Major Supercharger increases are underway for ...,https://twitter.com/elonmusk/status/1276957305...,2020-06-27 19:15:21+00:00
0,elonmusk,1276959792876011520,GerberKawasaki,2020-06-27 19:25:14+00:00,475,6699,[],,"Physics favors electric transport, batteries f...",https://twitter.com/elonmusk/status/1276959792...,2020-06-27 19:25:14+00:00


# Looks like the API only has date as it's finest time setting.

### To handle this, we'll have to eliminate duplicates after joining the new tweet data.

In [ ]:
def scan_for_new_tweets(path,username="elonmusk"):
    """Scans for new tweets ad adds them to the end
        of the old data frame"""
    df_old = reload_tweet_data(path,username) #get the old tweets
    #look for new tweets starting from latest date
    df_new = scrape_new_tweets(df_old['Time'].max(),username)
    if df_new == None:# No new tweets
        return df_old
    else:
        df_combined = prepend_new_tweets(df_new,df_old)
        return df_combined

In [18]:
df_combined = prepend_new_tweets(test_tweets_df,new_tweets_df)

In [22]:
result = pd.concat([new_tweets_df,test_tweets_df]).reset_index().drop('index',axis='columns')
result.sort_values(by='Time',ascending=True,inplace=True)
result.tail(10)

,username,tweet_id,reply_to,date,retweets,favorites,hashtags,mentions,text,permalink,Time
236,elonmusk,1276716075422367746,waitbutwhy,2020-06-27 03:16:47+00:00,820,12922,[],,It’s a tough one. That’s why we should expand ...,https://twitter.com/elonmusk/status/1276716075...,2020-06-27 03:16:47+00:00
241,elonmusk,1276716075422367746,waitbutwhy,2020-06-27 03:16:47+00:00,820,12922,[],,It’s a tough one. That’s why we should expand ...,https://twitter.com/elonmusk/status/1276716075...,2020-06-27 03:16:47+00:00
237,elonmusk,1276738326804873217,slashdot,2020-06-27 04:45:12+00:00,136,2263,[],,"Verified should be far more widespread, simply...",https://twitter.com/elonmusk/status/1276738326...,2020-06-27 04:45:12+00:00
242,elonmusk,1276738326804873217,slashdot,2020-06-27 04:45:12+00:00,136,2263,[],,"Verified should be far more widespread, simply...",https://twitter.com/elonmusk/status/1276738326...,2020-06-27 04:45:12+00:00
238,elonmusk,1276747755914842112,CathieDWood,2020-06-27 05:22:40+00:00,104,2713,[],,For sure,https://twitter.com/elonmusk/status/1276747755...,2020-06-27 05:22:40+00:00
243,elonmusk,1276747755914842112,CathieDWood,2020-06-27 05:22:40+00:00,104,2713,[],,For sure,https://twitter.com/elonmusk/status/1276747755...,2020-06-27 05:22:40+00:00
239,elonmusk,1276957305842528256,BLKMDL3,2020-06-27 19:15:21+00:00,324,15549,[],,Major Supercharger increases are underway for ...,https://twitter.com/elonmusk/status/1276957305...,2020-06-27 19:15:21+00:00
244,elonmusk,1276957305842528256,BLKMDL3,2020-06-27 19:15:21+00:00,324,15550,[],,Major Supercharger increases are underway for ...,https://twitter.com/elonmusk/status/1276957305...,2020-06-27 19:15:21+00:00
240,elonmusk,1276959792876011520,GerberKawasaki,2020-06-27 19:25:14+00:00,475,6698,[],,"Physics favors electric transport, batteries f...",https://twitter.com/elonmusk/status/1276959792...,2020-06-27 19:25:14+00:00
245,elonmusk,1276959792876011520,GerberKawasaki,2020-06-27 19:25:14+00:00,475,6699,[],,"Physics favors electric transport, batteries f...",https://twitter.com/elonmusk/status/1276959792...,2020-06-27 19:25:14+00:00
